## 🎨🧩 Step 1: Public Artworks Data Modeling

### Import Libraries

In [1]:
# Import Libraries
import osmnx as ox # to fetch data from OpenStreetMap
import geopandas as gpd # to work with geospatial data
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

### Create the tag to pull galleries from tourism section

In [2]:
tags = {
        "tourism": "artwork"
       }

### Fetch Berlin Geometries

In [6]:
public_artworks_raw = ox.features_from_place("Berlin, Germany", tags)


### Display basic info

In [7]:
print(f"Number of artworks entries fetched: {len(public_artworks_raw)}")
public_artworks_raw.head()

Number of artworks entries fetched: 2607


geometry artwork_type material  \
element id                                                           
node    28341970   POINT (13.62509 52.44417)    sculpture    metal   
        243487615  POINT (13.36187 52.51007)       statue      NaN   
        255049659  POINT (13.16499 52.43388)       statue      NaN   
        258485628  POINT (13.47316 52.52873)    sculpture    stone   
        262455591  POINT (13.39274 52.51726)       statue      NaN   

                                                        name  tourism  \
element id                                                              
node    28341970                                Seepferdchen  artwork   
        243487615                             Richard Wagner  artwork   
        255049659                           Flensburger Löwe  artwork   
        258485628                                  Begegnung  artwork   
        262455591  Reiterstandbild Friedrich II. von Preußen  artwork   

                                                               image  \
element id                                                             
node    28341970                                                 NaN   
        243487615        https://photos.app.goo.gl/9VgMjZd9uThAhJAb8   
        255049659                                                NaN   
        258485628  https://fennpfuhl.digital/img/statue/begegnung...   
        262455591  https://commons.wikimedia.org/wiki/File:Berlin...   

                  wheelchair    wikidata  \
element id                                 
node    28341970         NaN         NaN   
        243487615        yes    Q2148898   
        255049659    limited  Q105045191   
        258485628         no  Q110311378   
        262455591        yes     Q881611   

                                                  wikipedia  \
element id                                                    
node    28341970                                        NaN   
        243487615                en:Richard Wagner Monument   
        255049659                                       NaN   
        258485628                                       NaN   
        262455591  de:Reiterstandbild Friedrichs des Großen   

                                         artist_name  ... website:de  \
element id                                            ...              
node    28341970                                 NaN  ...        NaN   
        243487615                                NaN  ...        NaN   
        255049659  Kopie nach Hermann Wilhelm Bissen  ...        NaN   
        258485628                           Jo Doese  ...        NaN   
        262455591             Christian Daniel Rauch  ...        NaN   

                  geoglyph:language handrail step_count tactile_paving  \
element id                                                               
node    28341970                NaN      NaN        NaN            NaN   
        243487615               NaN      NaN        NaN            NaN   
        255049659               NaN      NaN        NaN            NaN   
        258485628               NaN      NaN        NaN            NaN   
        262455591               NaN      NaN        NaN            NaN   

                  tactile_writing:braille:de  \
element id                                     
node    28341970                         NaN   
        243487615                        NaN   
        255049659                        NaN   
        258485628                        NaN   
        262455591                        NaN   

                  tactile_writing:embossed_printed_letters:de building:colour  \
element id                                                                      
node    28341970                                          NaN             NaN   
        243487615                                         NaN             NaN   
        255049659                                         NaN             NaN   
        258

### Save the raw data to a csv file

- Define file paths

In [71]:
raw_csv_path = "../sources/csv_files/public_artwork_raw.csv"
raw_geojson_path = "../sources/geojson_files/public_artwork_raw.geojson"

- Save csv & geojson to correct folders in sources

In [72]:
public_artworks_raw.to_csv(raw_csv_path, index=False )

In [73]:
public_artworks_raw.to_file(raw_geojson_path, driver="GeoJSON")

### Remove the columns with 75% or more missing data

In [14]:
# Step 1: Calculate percentage of missing values per column
missing_percent = public_artworks_raw.isnull().mean() * 100

# Step 2: Identify columns with less than 75% missing
columns_to_keep = missing_percent[missing_percent < 75].index

# Step 3: Create a new filtered DataFrame
public_artworks_filtered = public_artworks_raw[columns_to_keep]

# Check shape and preview
print(public_artworks_filtered.shape)
public_artworks_filtered.head()

(2607, 10)


geometry artwork_type material  \
element id                                                           
node    28341970   POINT (13.62509 52.44417)    sculpture    metal   
        243487615  POINT (13.36187 52.51007)       statue      NaN   
        255049659  POINT (13.16499 52.43388)       statue      NaN   
        258485628  POINT (13.47316 52.52873)    sculpture    stone   
        262455591  POINT (13.39274 52.51726)       statue      NaN   

                                                        name  tourism  \
element id                                                              
node    28341970                                Seepferdchen  artwork   
        243487615                             Richard Wagner  artwork   
        255049659                           Flensburger Löwe  artwork   
        258485628                                  Begegnung  artwork   
        262455591  Reiterstandbild Friedrich II. von Preußen  artwork   

                                                               image  \
element id                                                             
node    28341970                                                 NaN   
        243487615        https://photos.app.goo.gl/9VgMjZd9uThAhJAb8   
        255049659                                                NaN   
        258485628  https://fennpfuhl.digital/img/statue/begegnung...   
        262455591  https://commons.wikimedia.org/wiki/File:Berlin...   

                     wikidata                        artist_name start_date  \
element id                                                                    
node    28341970          NaN                                NaN        NaN   
        243487615    Q2148898                                NaN        NaN   
        255049659  Q105045191  Kopie nach Hermann Wilhelm Bissen       1938   
        258485628  Q110311378                           Jo Doese        NaN   
        262455591     Q881611             Christian Daniel Rauch        NaN   

                                                   wikimedia_commons  
element id                                                            
node    28341970                                                 NaN  
        243487615                                                NaN  
        255049659           Category:Flensburg Lion (Copy in Berlin)  
        258485628                 Category:Begegnung (Joachim Doese)  
        262455591  Category:Reiterstandbild Friedrichs des Großen...

### Check no Geometries missing

In [15]:
print("Missing geometries:", public_artworks_filtered.geometry.isna().sum())

Missing geometries: 0


### Add Latitude & Longitude columns to filtered geodataframe

In [16]:
# Reproject and extract lat/lon
public_artworks_filtered = public_artworks_filtered.to_crs(epsg=4326)
public_artworks_filtered['geometry'] = public_artworks_filtered['geometry'].apply(
    lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point()
)
public_artworks_filtered['latitude'] = public_artworks_filtered.geometry.y
public_artworks_filtered['longitude'] = public_artworks_filtered.geometry.x

### Verify the lat/lon amounts are correct

In [17]:
print("Latitude range:", public_artworks_filtered["latitude"].min(), "to", public_artworks_filtered["latitude"].max())

print("Longitude range:", public_artworks_filtered["longitude"].min(), "to", public_artworks_filtered["longitude"].max())

Latitude range: 52.3975036 to 52.669101
Longitude range: 13.0913224 to 13.7208069


### Reset index and change column name 'id' to 'public_artworks_id'

In [18]:
public_artworks_filtered = public_artworks_filtered.reset_index()

# Rename the "id" column to "public_artwork_id"
public_artworks_filtered = public_artworks_filtered.rename(columns={"id": "public_artwork_id"})  
# set the public_artwork_id to string
public_artworks_filtered["public_artwork_id"] = public_artworks_filtered["public_artwork_id"].astype(str)
#  
# Drop the redundant column "element"
public_artworks_filtered= public_artworks_filtered.drop(columns=["element"],errors='ignore')
# Show final list of columns
print(public_artworks_filtered.columns.tolist())

['public_artwork_id', 'geometry', 'artwork_type', 'material', 'name', 'tourism', 'image', 'wikidata', 'artist_name', 'start_date', 'wikimedia_commons', 'latitude', 'longitude']


In [19]:
public_artworks_filtered.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2607 entries, 0 to 2606
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   public_artwork_id  2607 non-null   object  
 1   geometry           2607 non-null   geometry
 2   artwork_type       2401 non-null   object  
 3   material           1184 non-null   object  
 4   name               1775 non-null   object  
 5   tourism            2607 non-null   object  
 6   image              715 non-null    object  
 7   wikidata           1173 non-null   object  
 8   artist_name        1466 non-null   object  
 9   start_date         936 non-null    object  
 10  wikimedia_commons  824 non-null    object  
 11  latitude           2607 non-null   float64 
 12  longitude          2607 non-null   float64 
dtypes: float64(2), geometry(1), object(10)
memory usage: 264.9+ KB


---

## 🎨🔄 Step 2: Public Artworks Data Transformation

### Drop unnecessary columns
    - Dont need Berlin and DE as user knows they are looking for data in Berlin, Germany
    - Drop tourism column as all artworks
    - Drop suburb as it is the same as neighborhood which will be added later to make sure nothing is missing

In [ ]:
public_artworks_filtered.drop(columns=['addr:city', 'addr:country', 'tourism', 'addr:suburb'], errors='ignore', inplace=True)

### Copy the filtered database for record purposes and then standardise column names

In [21]:
public_artworks_cleaned = public_artworks_filtered.copy()

def clean_column(col):
    col = col.strip().lower().replace('addr:', '')
    col = col.replace(' ', '_')
    col = re.sub(r'[^a-z0-9_]', '', col)
    return col

public_artworks_cleaned.columns = [clean_column(col) for col in public_artworks_cleaned.columns]

### Rename columns for clarity

In [23]:
public_artworks_cleaned.rename(columns={
    'name': 'artwork_name'
}, inplace=True)

In [24]:
public_artworks_cleaned.columns.tolist()

['public_artwork_id',
 'geometry',
 'artwork_type',
 'material',
 'artwork_name',
 'image',
 'wikidata',
 'artist_name',
 'start_date',
 'wikimedia_commons',
 'latitude',
 'longitude']

### Fetch Districts & Neighbourhoods

- Load official Berlin districts GeoDataFrame

In [25]:
districts_gdf = gpd.read_file("../sources/geojson_files/lor_ortsteile.geojson")

In [26]:
districts_gdf.head(2)

,gml_id,spatial_name,spatial_alias,spatial_type,OTEIL,BEZIRK,FLAECHE_HA,geometry
0,re_ortsteil.0101,0101,Mitte,Polygon,Mitte,Mitte,1063.8748,"POLYGON ((13.41649 52.52696, 13.41635 52.52702..."
1,re_ortsteil.0102,0102,Moabit,Polygon,Moabit,Mitte,768.7909,"POLYGON ((13.33884 52.51974, 13.33884 52.51974..."


- Reproject GeoDataFrames to EPSG:4326 

In [27]:
public_artworks_cleaned = public_artworks_cleaned.to_crs(epsg=4326)
districts_gdf = districts_gdf.to_crs(epsg=4326)

- Spatial join with district(Bezirk) and Neighbourhood_id(spatial_name)

In [29]:
public_artworks_df_district = gpd.sjoin(
    public_artworks_cleaned,
    districts_gdf[["BEZIRK", "spatial_name","geometry"]],
    how="left",
    predicate="within"
)

- Rename columns for clarity

In [30]:

public_artworks_df_district = public_artworks_df_district.rename(columns={
    "BEZIRK": "district",
    "spatial_name": "neighbourhood_id"
}).drop(columns=["index_right"])  

- District mapping (official codes as strings)

In [31]:
district_mapping = {
    'Mitte': '11001001',
    'Friedrichshain-Kreuzberg': '11002002',
    'Pankow': '11003003',
    'Charlottenburg-Wilmersdorf': '11004004',
    'Spandau': '11005005',
    'Steglitz-Zehlendorf': '11006006',
    'Tempelhof-Schöneberg': '11007007',
    'Neukölln': '11008008',
    'Treptow-Köpenick': '11009009',
    'Marzahn-Hellersdorf': '11010010',
    'Lichtenberg': '11011011',
    'Reinickendorf': '11012012'
}

# Apply mapping to create district_id column (string)
public_artworks_df_district['district_id'] = public_artworks_df_district['district'].map(district_mapping).astype(str)

In [32]:
public_artworks_df_district.head()

,public_artwork_id,geometry,artwork_type,material,artwork_name,image,wikidata,artist_name,start_date,wikimedia_commons,latitude,longitude,district,neighbourhood_id,district_id
0,28341970,POINT (13.62509 52.44417),sculpture,metal,Seepferdchen,NaN,NaN,NaN,NaN,NaN,52.444170,13.625088,Treptow-Köpenick,0910,11009009
1,243487615,POINT (13.36187 52.51007),statue,NaN,Richard Wagner,https://photos.app.goo.gl/9VgMjZd9uThAhJAb8,Q2148898,NaN,NaN,NaN,52.510066,13.361869,Mitte,0104,11001001
2,255049659,POINT (13.16499 52.43388),statue,NaN,Flensburger Löwe,NaN,Q105045191,Kopie nach Hermann Wilhelm Bissen,1938,Category:Flensburg Lion (Copy in Berlin),52.433884,13.164987,Steglitz-Zehlendorf,0607,11006006
3,258485628,POINT (13.47316 52.52873),sculpture,stone,Begegnung,https://fennpfuhl.digital/img/statue/begegnung...,Q110311378,Jo Doese,NaN,Category:Begegnung (Joachim Doese),52.528735,13.473156,Lichtenberg,1111,11011011
4,262455591,POINT (13.39274 52.51726),statue,NaN,Reiterstandbild Friedrich II. von Preußen,https://commons.wikimedia.org/wiki/File:Berlin...,Q881611,Christian Daniel Rauch,NaN,Category:Reiterstandbild Friedrichs des Großen...,52.517260,13.392744,Mitte,0101,11001001


### Used reverse Geocoding and Nominatim to get Postal code, Street & house number info
    - Created postal_code, street and house_number columns to try get data from the latitude and longitude

In [33]:
tqdm.pandas()

# Initialize geocoder
geolocator = Nominatim(user_agent="gallery_locator")

# Define function to extract postal code and street
def get_postcode_and_street(row):
    try:
        location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
        address = location.raw.get('address', {})
        postcode = address.get('postcode')
        street = address.get('road') or address.get('pedestrian') or address.get('footway') or address.get('street')
        house_number = address.get('housenumber')
        return pd.Series([postcode, street, house_number])
    except:
        return pd.Series([None, None, None])

# Apply to your DataFrame
public_artworks_df_district[['postal_code', 'street', 'house_number']] = public_artworks_df_district.progress_apply(get_postcode_and_street, axis=1)


100%|██████████| 2607/2607 [1:10:39<00:00,  1.63s/it]   


In [35]:
public_artworks_full = public_artworks_df_district.copy()

### Data Cleaning & Normalization
    - Replace all missing values with Nan

In [36]:
# Replace empty strings with NaN
public_artworks_full.replace('', np.nan, inplace=True)

public_artworks_full.head(2)

,public_artwork_id,geometry,artwork_type,material,artwork_name,image,wikidata,artist_name,start_date,wikimedia_commons,latitude,longitude,district,neighbourhood_id,district_id,postal_code,street,house_number
0,28341970,POINT (13.62509 52.44417),sculpture,metal,Seepferdchen,NaN,NaN,NaN,NaN,NaN,52.444170,13.625088,Treptow-Köpenick,0910,11009009,12559,Spreetunnel,None
1,243487615,POINT (13.36187 52.51007),statue,NaN,Richard Wagner,https://photos.app.goo.gl/9VgMjZd9uThAhJAb8,Q2148898,NaN,NaN,NaN,52.510066,13.361869,Mitte,0104,11001001,10785,Tiergartenstraße,None


### Normalize the street name column

In [96]:
def normalize_street_name(name):
    if pd.isna(name):
        return np.nan
    # Replace underscores with spaces
    name = name.replace('_', ' ').replace('-', ' ')
    # Replace 'str.' or 'str' at end with ' Straße'
    name = re.sub(r'\bstr\.?\s*$', ' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'straße' if missing
    name = re.sub(r'(?<!\s)(straße)$', r' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'allee' if missing
    name = re.sub(r'(?<!\s)(allee)$', r' Allee', name, flags=re.IGNORECASE)
    # Ensure space before 'damm' if missing
    name = re.sub(r'(?<!\s)(damm)$', r' Damm', name, flags=re.IGNORECASE)
    # Ensure space before 'weg' if missing
    name = re.sub(r'(?<!\s)(weg)$', r' Weg', name, flags=re.IGNORECASE)
    # Ensure space before 'graben' if missing
    name = re.sub(r'(?<!\s)(graben)$', r' Graben', name, flags=re.IGNORECASE)
    # Ensure space before 'ufer' if missing
    name = re.sub(r'(?<!\s)(ufer)$', r' Ufer', name, flags=re.IGNORECASE)
    # Ensure space before 'korso' if missing
    name = re.sub(r'(?<!\s)(korso)$', r' Korso', name, flags=re.IGNORECASE)
    # Ensure space before 'zeile' if missing
    name = re.sub(r'(?<!\s)(zeile)$', r' Ziele', name, flags=re.IGNORECASE)
    # Ensure space before 'promenade' if missing
    name = re.sub(r'(?<!\s)(promenade)$', r' Promenade', name, flags=re.IGNORECASE)
    # Ensure space before 'kiez' if missing
    name = re.sub(r'(?<!\s)(kiez)$', r' Kiez', name, flags=re.IGNORECASE)
    # Ensure space before 'platz' if missing
    name = re.sub(r'(?<!\s)(platz)$', r' Platz', name, flags=re.IGNORECASE)
    # Ensure space before 'steig' if missing
    name = re.sub(r'(?<!\s)(steig)$', r' Steig', name, flags=re.IGNORECASE)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [97]:
public_artworks_full['street'] = public_artworks_full['street'].apply(normalize_street_name)

### Drop unnecessary columns

In [98]:
public_artworks_full.drop(columns=['geometry', 'district'], errors='ignore', inplace=True)

### Convert all text in columns to lowercase to avoid any duplications

In [99]:
text_cols = ["artwork_type", "material", "artwork_name", "image", "artist_name", "wikimedia_commons", "street"]

for col in text_cols:
    if col in public_artworks_full.columns:
        public_artworks_full[col] = public_artworks_full[col].apply(
            lambda x: x.strip().lower() if isinstance(x, str) else x
        )

### Check column data types
    - All have correct type allocated so no need to change anything

In [100]:
public_artworks_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2607 entries, 0 to 2606
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   public_artwork_id  2607 non-null   object 
 1   artwork_type       2401 non-null   object 
 2   material           1184 non-null   object 
 3   artwork_name       1775 non-null   object 
 4   image              715 non-null    object 
 5   wikidata           1173 non-null   object 
 6   artist_name        1466 non-null   object 
 7   start_date         936 non-null    object 
 8   wikimedia_commons  824 non-null    object 
 9   latitude           2607 non-null   float64
 10  longitude          2607 non-null   float64
 11  neighbourhood_id   2607 non-null   object 
 12  district_id        2607 non-null   object 
 13  postal_code        2544 non-null   object 
 14  street             2541 non-null   object 
 15  house_number       0 non-null      object 
dtypes: float64(2), object(14

### Remove duplicates
    - In this case we have no duplicates but will keep this check for future runs incase

In [101]:
# See how many duplicate rows exist
public_artworks_full.duplicated().sum()

np.int64(0)

In [102]:
# Display the actual duplicate rows
public_artworks_full[public_artworks_full.duplicated()]

,public_artwork_id,artwork_type,material,artwork_name,image,wikidata,artist_name,start_date,wikimedia_commons,latitude,longitude,neighbourhood_id,district_id,postal_code,street,house_number


In [ ]:
# Remove duplicate rows and reset index
public_artworks_full = public_artworks_full.drop_duplicates().reset_index(drop=True)

### Remove row if name missing

In [116]:
public_artworks_full = public_artworks_full.dropna(subset=['artwork_name'])

### Reorder column names to be clearer

In [117]:
public_artworks_listings = public_artworks_full[['public_artwork_id', 'artwork_name', 'artwork_type', 'artist_name', 'street', 'neighbourhood_id', 
                                                 'district_id', 'postal_code', 'material', 'start_date','image', 'wikidata',  'wikimedia_commons', 'latitude', 'longitude']]

In [118]:
public_artworks_listings.to_csv("../sources/csv_files/public_artwork_listings.csv", index=False)

### Final Summary of cleaned and Transformed Data

In [119]:
# Shape of dataframe
print(f"Number of rows: {public_artworks_listings.shape[0]}")
print(f"Number of columns: {public_artworks_listings.shape[1]}")

Number of rows: 1775
Number of columns: 15


In [120]:
# Column list
print("\nRemaining columns:")
print(public_artworks_listings.columns.tolist())


Remaining columns:
['public_artwork_id', 'artwork_name', 'artwork_type', 'artist_name', 'street', 'neighbourhood_id', 'district_id', 'postal_code', 'material', 'start_date', 'image', 'wikidata', 'wikimedia_commons', 'latitude', 'longitude']


In [121]:
# Missing values check
missing = public_artworks_listings.isnull().sum()
print("\nMissing values after cleaning and transforming :")
print(missing)


Missing values after cleaning and transforming :
public_artwork_id       0
artwork_name            0
artwork_type           56
artist_name           428
street                 44
neighbourhood_id        0
district_id             0
postal_code            42
material              781
start_date            922
image                1259
wikidata              698
wikimedia_commons    1079
latitude                0
longitude               0
dtype: int64


In [122]:
# Material Types Frequency
material_counts = public_artworks_listings['material'].value_counts()
material_counts

material
bronze                              369
sandstone                           105
stone                               103
steel                                72
metal                                39
                                   ... 
stainless steel, synthetic resin      1
beton                                 1
sandstone;granite                     1
marble, steel                         1
metal_sheet                           1
Name: count, Length: 126, dtype: int64

In [123]:
# Artwork Types Frequency
artwork_type_counts = public_artworks_listings['artwork_type'].value_counts()
artwork_type_counts

artwork_type
sculpture          858
statue             522
graffiti           102
mural               85
installation        83
relief              25
bust                 8
architecture         7
mosaic               6
stone                5
ensemble             4
stele                4
fountain             2
painting             2
print                1
land_art             1
discs                1
plaque               1
sculpture_brick      1
column               1
Name: count, dtype: int64

In [124]:
# Artwork per district
artworks_per_district = public_artworks_listings['district_id'].value_counts()
artworks_per_district

district_id
11001001    367
11009009    302
11011011    226
11002002    204
11004004    174
11010010    124
11003003    123
11008008     87
11006006     60
11007007     45
11005005     35
11012012     28
Name: count, dtype: int64

In [125]:
# Show first and last date of artworks
public_artworks_listings['start_date'].sort_values()

1150          1647
111           1740
357           1791
382           1793
30      1827..1834
           ...    
2575           NaN
2576           NaN
2580           NaN
2581           NaN
2582           NaN
Name: start_date, Length: 1775, dtype: object

In [126]:
public_artworks_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1775 entries, 0 to 2588
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   public_artwork_id  1775 non-null   object 
 1   artwork_name       1775 non-null   object 
 2   artwork_type       1719 non-null   object 
 3   artist_name        1347 non-null   object 
 4   street             1731 non-null   object 
 5   neighbourhood_id   1775 non-null   object 
 6   district_id        1775 non-null   object 
 7   postal_code        1733 non-null   object 
 8   material           994 non-null    object 
 9   start_date         853 non-null    object 
 10  image              516 non-null    object 
 11  wikidata           1077 non-null   object 
 12  wikimedia_commons  696 non-null    object 
 13  latitude           1775 non-null   float64
 14  longitude          1775 non-null   float64
dtypes: float64(2), object(13)
memory usage: 221.9+ KB


In [127]:
public_artworks_listings.head()

,public_artwork_id,artwork_name,artwork_type,artist_name,street,neighbourhood_id,district_id,postal_code,material,start_date,image,wikidata,wikimedia_commons,latitude,longitude
0,28341970,seepferdchen,sculpture,NaN,spreetunnel,0910,11009009,12559,metal,NaN,NaN,NaN,NaN,52.444170,13.625088
1,243487615,richard wagner,statue,NaN,tiergarten straße,0104,11001001,10785,NaN,NaN,https://photos.app.goo.gl/9vgmjzd9uthahjab8,Q2148898,NaN,52.510066,13.361869
2,255049659,flensburger löwe,statue,kopie nach hermann wilhelm bissen,tiefhorn weg,0607,11006006,14109,NaN,1938,NaN,Q105045191,category:flensburg lion (copy in berlin),52.433884,13.164987
3,258485628,begegnung,sculpture,jo doese,anton saefkow platz,1111,11011011,10369,stone,NaN,https://fennpfuhl.digital/img/statue/begegnung...,Q110311378,category:begegnung (joachim doese),52.528735,13.473156
4,262455591,reiterstandbild friedrich ii. von preußen,statue,christian daniel rauch,unter den linden,0101,11001001,10117,NaN,NaN,https://commons.wikimedia.org/wiki/file:berlin...,Q881611,category:reiterstandbild friedrichs des großen...,52.517260,13.392744
